# HW4 : A visit to the movie zoo!

![](https://vignette.wikia.nocookie.net/bojackhorseman/images/f/f2/HSACWDTK%3FDTKT%3FLFO%21%21.png/revision/latest?cb=20150720050503)

## Task
In this homework, your task is to visualize THREE non-typical charts on anything related to your favorite **movie star!**
This means you CANNOT use the Big 4 chart types or their close variants (i.e. Pie, Bar, Line and Scatter, Area, etc.)

You are free to use any other chart type whether or not they were covered in class!
The lecture on Visit To The Zoo is a good place to start to get ideas on what kinds of charts exist.

For the data, you are free to use any data source you deem fit.
For charting, we will NOT be constraining the technology you use. 
You are free to produce the charts in any way you would like.

You will be judged on
* Creativity
* Presentation Quality
* Data Quality (Did your visualization reveal something interesting?)

For extra credit, you can make a fully interactive visualization.

## Ideas for Data Collection

Here, we show an example of how to collect data about Arnold Schwarzenegger!
Do note that this is just an example of the kind of data you can collect.
You are **NOT** constrained
* To the same movie star (you can pick your own!)
* To the same *kind* of data
* To the same data sources
* or to anything else!

This assignment gives you the power to do what you like!

In [6]:
from imdbpie import ImdbFacade
from IPython.core.display import display, HTML
from bs4 import BeautifulSoup
import urllib.request
import re

#### Get data for Arnold Schwarzenegger

In [223]:
# Get an instance of IMDb class
imdb = ImdbFacade()
# Search for Arnold Schwarzenegger
people = imdb.search_for_name('Leonardo DiCaprio')
print(people)

(NameSearchResult(imdb_id='nm0000138', name='Leonardo DiCaprio'),)


In [225]:
# Fetch information about him
leonardo = imdb.get_name(people[0].imdb_id)
# What information do I have about him?
print('\n'.join([x for x in dir(leonardo) if not x.startswith('__')]))

bios
birth_place
date_of_birth
filmography
gender
image
imdb_id
name


In [13]:
# How many movies does he have?
print(len(leonardo.filmography))

430


In [14]:
# Let's fetch some more information about a movie
movie = imdb.get_title(leonardo.filmography[-1])

In [24]:
# What information can I get about this movie?
print('\n'.join([x for x in dir(movie) if not x.startswith('__')]))

certification
creators
credits
directors
episode
episodes
genres
image
imdb_id
plot_outline
rating
rating_count
release_date
releases
runtime
season
stars
title
type
writers
year


In [239]:
movie.directors

(TitleName(name='Leila Conners', category='director', imdb_id='nm2126446', job=None, characters=()),
 TitleName(name='Nadia Conners', category='director', imdb_id='nm2105775', job=None, characters=()))

In [16]:
print(movie.imdb_id)

tt0492931


In [18]:
html = """
    <div style="background-color:#FFDDDD">
    <h2> Warning! </h2>
    <p> This code below is meant to be an example of what you can do. <br>
        It is not guaranteed to work always, and will need to be tweaked!
    </p>
    </div>
"""
display(HTML(html))

#### Box office numbers

In [34]:
# Let's experiment with Terminator
imdb_id = 'tt0492931'

# Fetch the box office numbers
base = 'https://www.boxofficemojo.com'
url = base + '/title/' + imdb_id
source = urllib.request.urlopen(url).read()
soup = BeautifulSoup(source,'lxml')

table = soup('th', text=re.compile(r'Release Group'))[0].parent.parent
group = table.findAll('tr', recursive=False)[1].find('a').get('href')
url = base + group

# Get total earnings domestic and international
source = urllib.request.urlopen(url).read()
soup = BeautifulSoup(source,'lxml')
earnings = soup('h2', text=re.compile(r'Rollout'))[0].parent.parent.findAll('div')
domestic = earnings[1].find('span', {'class': 'money'}).get_text()
domestic_url = earnings[1].find('a').get('href')
international = earnings[2].find('span', {'class': 'money'}).get_text()

# Get weekly domestic earnings
url = base + domestic_url
url = url[:url.rfind('/')] + '/weekly/'
source = urllib.request.urlopen(url).read()
soup = BeautifulSoup(source,'lxml')
table = soup.find('div', {'class':'a-section imdb-scroll-table-inner'}).findAll('tr')
weekly = []
for tr in table[1:]:
    date = tr.findAll('td')[0].get_text()
    earning = tr.findAll('td')[2].get_text()
    weekly.append((date, earning))

# Print the values we've just got!
print("Total Domestic Earnings: %s" % domestic)
print("Total International Earnings: %s" % international)
print("Weekly Domestic Earnings:")
for date, earning in weekly:
    print("\t%s \t: %s" % (date, earning))

Total Domestic Earnings: $707,343
Total International Earnings: $277,864
Weekly Domestic Earnings:
	Aug 17-23 	: $80,413
	Aug 24-30 	: $97,507
	Aug 31-Sep 6 	: $298,393
	Sep 7-13 	: $117,167
	Sep 14-20 	: $77,674
	Sep 21-27 	: $26,623
	Sep 28-Oct 4 	: $9,566


## Scrape Major movie starred by Leonardo ( Major movies are those having a budget)

In [275]:
url = 'https://pro.imdb.com/name/nm0000138/filmography'
source = urllib.request.urlopen(url).read()
soup = BeautifulSoup(source,'lxml')
table = soup.find("table",{"id":"past_film_filmography"})

In [293]:
rows = table.findAll("tr")
films_info = []
for i in range(1,len(rows)):
    row = rows[i]
    imdb_id = row['data-filter-item-id']
    
    cols = row.findAll('td')
    try:
        budget = cols[1]['data-sort-value']
        opening_weekend = cols[2]['data-sort-value']
        gross_US_Canada = cols[3]['data-sort-value']
        gross_world = cols[4]['data-sort-value']
        films_info.append([imdb_id, budget, opening_weekend, gross_US_Canada, gross_world])
    except KeyError:
        continue

In [295]:
films_info_df = pd.DataFrame(columns=['IMDB_id', 'budget', 'opening_weekend', 'gross_US_Canada', 'gross_world'], 
                             data = films_info)

In [300]:
movie_names = films_info_df.apply(lambda x: imdb.get_title(x['IMDB_id']).title, axis = 1)

In [301]:
films_info_df['movie_name'] = movie_names

In [303]:
films_info_df['year'] = films_info_df.apply(lambda x: imdb.get_title(x['IMDB_id']).year, axis = 1)

In [432]:
selected = films_info_df[['year', 'gross_world', 'movie_name']]

In [436]:
selected = selected.rename({'gross_world':'size', 'movie_name':'text'}, axis = 1)

In [439]:
selected.size = selected['size'].apply(lambda x: int(x))

In [441]:
selected['size'][0]

371544347

In [443]:
selected['category'] = ''

In [449]:
selected

,year,size,text,category
0,2019,371544347,Once Upon a Time... in Hollywood,
1,2018,85878203,Robin Hood,
2,2016,22678555,Live by Night,
3,2015,532950503,The Revenant,
4,2013,392000694,The Wolf of Wall Street,
5,2013,15435261,Out of the Furnace,
6,2013,62675095,Runner Runner,
7,2013,28277704,Don Jon,
8,2013,353641895,The Great Gatsby,
9,2012,425368238,Django Unchained,


In [446]:
selected.to_json(orient='records')

'[{"year":2019,"size":371544347,"text":"Once Upon a Time... in Hollywood","category":""},{"year":2018,"size":85878203,"text":"Robin Hood","category":""},{"year":2016,"size":22678555,"text":"Live by Night","category":""},{"year":2015,"size":532950503,"text":"The Revenant","category":""},{"year":2013,"size":392000694,"text":"The Wolf of Wall Street","category":""},{"year":2013,"size":15435261,"text":"Out of the Furnace","category":""},{"year":2013,"size":62675095,"text":"Runner Runner","category":""},{"year":2013,"size":28277704,"text":"Don Jon","category":""},{"year":2013,"size":353641895,"text":"The Great Gatsby","category":""},{"year":2012,"size":425368238,"text":"Django Unchained","category":""},{"year":2011,"size":84920539,"text":"J. Edgar","category":""},{"year":2011,"size":75993061,"text":"The Ides of March","category":""},{"year":2011,"size":89162162,"text":"Red Riding Hood","category":""},{"year":2010,"size":829895144,"text":"Inception","category":""},{"year":2010,"size":2948041

## Scrape Global Sales of 2012 Release of Titanic

In [410]:
## Scrape Global Sales of 2012 Release of Titanic
url = 'https://www.boxofficemojo.com/releasegroup/gr4211823109/'
source = urllib.request.urlopen(url).read()
soup = BeautifulSoup(source,'lxml')
tables = soup.findAll("table",{"class":"a-bordered a-horizontal-stripes mojo-table releases-by-region"})
box_info_2012 = []
for table in tables:
    rows = table.findAll('tr')
    for i in range(2,len(rows)):
        row = rows[i]
        market = row.find("a").getText()
        release_date = row.findAll('td')[1].getText()
        gross = row.findAll('td')[-1].getText()
        box_info_2012.append([market, release_date, gross])

In [416]:
box_info_2012_df = pd.DataFrame(box_info_2012, columns=['Market', 'Release', 'box_office'])
box_info_2012_df['box_office'] = box_info_2012_df['box_office'].apply(lambda x: "".join([c for c in x if c.isnumeric()]))
box_info_2012_df.iloc[0,0] = 'United States'
import pycountry
country_codes = []
for i in box_info_2012_df['Market'].to_list():
    if '/' in i:
        i = i[i.find('/')+1:]
    try:
        country_codes.append(pycountry.countries.get(name = i).alpha_2)
    except:
        country_codes.append('')

In [420]:
country_codes[6] = 'CZ'
country_codes[23] = 'CS'
country_codes[34] = 'BO'
country_codes[42] = 'VE'
country_codes[51] = 'RU'
country_codes[53] = 'KR'

In [422]:
box_info_2012_df['market_code'] = country_codes
market_dict = box_info_2012_df[['market_code', 'box_office']].set_index('market_code').iloc[:,0].to_dict()
market_dict = pd.Series(market_dict).apply(lambda x: int(x)).to_dict()


In [430]:
sum(market_dict.values())

335148116

In [424]:
import json
with open('market_box_info_2012.json', 'w') as fp:
    json.dump(market_dict, fp)

## Scrape Global Sales of Original Release of Titanic

In [242]:
url = 'https://www.boxofficemojo.com/releasegroup/gr4195045893/'
source = urllib.request.urlopen(url).read()
soup = BeautifulSoup(source,'lxml')

In [245]:
tables = soup.findAll("table",{"class":"a-bordered a-horizontal-stripes mojo-table releases-by-region"})

In [266]:
box_info_1998 = []
for table in tables:
    rows = table.findAll('tr')
    for i in range(2,len(rows)):
        row = rows[i]
        market = row.find("a").getText()
        release_date = row.findAll('td')[1].getText()
        gross = row.findAll('td')[-1].getText()
        box_info_1998.append([market, release_date, gross])

In [268]:
box_info_1998[0]

['Domestic', 'Dec 19, 1997', '$600,788,188']

In [308]:
box_info_1998_df = pd.DataFrame(box_info_1998, columns=['Market', 'Release', 'box_office'])

In [312]:
box_info_1998_df['box_office'] = box_info_1998_df['box_office'].apply(lambda x: "".join([c for c in x if c.isnumeric()]))

In [333]:
box_info_1998_df.iloc[0,0] = 'United States'

In [386]:
import pycountry

In [360]:
country_codes = []
for i in box_info_1998_df['Market'].to_list():
    if '/' in i:
        i = i[i.find('/')+1:]
    try:
        country_codes.append(pycountry.countries.get(name = i).alpha_2)
    except:
        country_codes.append('')

In [380]:
country_codes[13] = 'GR'
country_codes[36] = 'YU'
country_codes[38] = 'BO'
country_codes[52] = 'VE'
country_codes[61] = 'RU'
country_codes[63] = 'KR'
country_codes[64] = 'TW'

In [382]:
box_info_1998_df['market_code'] = country_codes

In [383]:
market_dict = box_info_1998_df[['market_code', 'box_office']].set_index('market_code').iloc[:,0].to_dict()

In [402]:
market_dict = pd.Series(market_dict).apply(lambda x: int(x)).to_dict()

In [403]:
import json
with open('market_box_info.json', 'w') as fp:
    json.dump(market_dict, fp)

In [404]:
pd.Series(market_dict).sum()

1845646818

## Scrape Global Sales of 2012 3D Release of Titanic

In [269]:
url = 'https://www.boxofficemojo.com/releasegroup/gr4211823109/'
source = urllib.request.urlopen(url).read()
soup = BeautifulSoup(source,'lxml')
tables = soup.findAll("table",{"class":"a-bordered a-horizontal-stripes mojo-table releases-by-region"})
box_info_2012 = []
for table in tables:
    rows = table.findAll('tr')
    for i in range(2,len(rows)):
        row = rows[i]
        market = row.find("a").getText()
        release_date = row.findAll('td')[1].getText()
        gross = row.findAll('td')[-1].getText()
        box_info_2012.append([market, release_date, gross])

In [271]:
box_info_2012[0]

['Domestic', 'Apr 4, 2012', '$57,884,114']

## Scrape Scrips of Titanic

In [44]:
url = "https://www.imsdb.com/scripts/Titanic.html"
source = urllib.request.urlopen(url).read()
soup = BeautifulSoup(source,'lxml')

In [49]:
test = soup.findAll("pre")[0]

In [53]:
## Here is text of all lines in one string
all_text = text.get_text()

In [60]:
import pandas as pd

In [65]:
text_list = pd.Series(all_text.split("\n\n")).apply(lambda x: x.strip()).to_list()

In [ ]:
import string
punctuation = set(string.punctuation)

In [75]:
## check if the string is all capitalized after removing non-letter char: checking if it is a character
def all_Capitalized(string):
    return (''.join(x for x in string if x.isalpha())).isupper()

In [183]:
character_lines = dict()
character = ''
## only the first line is the actor's words 
## if starts with (, the next line is still actor's words
is_Word = True
for line in text_list:
    if line == '':
        continue
        
    if all_Capitalized(line):
        character = line
        is_Word = True
        if character not in character_lines.keys():
            character_lines[character] = []
    else:
        if line[0] == '(':
            is_Word = True
            continue
        if is_Word:
            character_lines[character].append(line)
            is_Word = False

In [184]:
character_freq = {}
for char in character_lines:
    character_freq[char] = len(character_lines[char])

In [185]:
character_freq_series = pd.Series(character_freq).sort_values(ascending = False)

In [186]:
## ten most frequent character
character_freq_series.head(10)

ROSE       228
JACK       195
CAL        103
LOVETT      58
MOLLY       31
BODINE      31
RUTH        30
SMITH       27
ANDREWS     27
LOVEJOY     22
dtype: int64

In [187]:
from nltk.corpus import stopwords
import string
punctuation = set(string.punctuation)
stopWordSet = set(stopwords.words("english"))
## clean the line by removing punctuation and stop words (i.e. noise words)
def clean_text(line):
    r = ''.join([c for c in line if not (c in punctuation)])
    r = ' '.join([word for word in r.split() if not (word.lower() in stopWordSet)])
    return r

In [188]:
cleaned_text = clean_text(all_text)

In [189]:
jack_text_series = pd.Series(character_lines['JACK']).apply(lambda line: clean_text(line))

In [190]:
jack_text = ' '.join(jack_text_series.to_list())

In [209]:
rose_text_series = pd.Series(character_lines['ROSE']).apply(lambda line: clean_text(line))
rose_text = ' '.join(rose_text_series.to_list())

In [200]:
cal_text_series = pd.Series(character_lines['CAL']).apply(lambda line: clean_text(line))
cal_text = ' '.join(cal_text_series.to_list())

In [220]:
open("cleaned_text.txt",'w').write(cleaned_text)

158464

In [221]:
open("rose_text.txt",'w').write(rose_text)

6792

In [222]:
open("jack_text.txt",'w').write(jack_text)

6158

## Other resources!

This assignment doesn't have a restriction on where you can look for data.
Further, we don't mind how you collect the data, or what data you collect.

Here are some additional resources for this example, and you can customize it for your own!
* Arnold Schwarzenegger Kill Count: https://www.youtube.com/watch?v=OE6jpTaOYMU
* Arnold Schwarzenegger Top Quotes: https://www.youtube.com/watch?v=pDxn0Xfqkgw

You could think about the IMDB network as a graph, with different actors connected through movies.

Some other useful libraries/ databases:
* IMDBPy
* http://www.omdbapi.com/